In [1]:
import types,time
from functools import wraps
class DNode:
    def __init__(self,key_ = None,value_=None):
        self.value = value_
        self.key = key_
        self.prev = None
        self.next = None

class lru_cache:

    def __init__(self, capacity: int):
        self.dict = dict()
        self.capacity = capacity
        self.size = 0
        self.head = DNode()
        self.tail = DNode()
        self.head.next = self.tail
        self.tail.prev = self.head
        
    def push(self, node):
        node.next = self.head.next
        node.prev = self.head
        self.head.next.prev = node
        self.head.next = node
        return node
        
    def remove(self, node):
        prev = node.prev
        nxt = node.next
        prev.next = nxt
        nxt.prev = prev
        
    def top(self, node):
        self.remove(node)
        self.push(node)
        
    def pop(self):
        k = self.tail.prev.key
        self.remove(self.tail.prev)
        return k
        
    def get(self, key: int) -> int:
        # print("______")
        # print("getting %d" % key)
        # print(self.dict.keys())
        if key not in self.dict:
            # print("not found!")
            return -1
        else:
            # print("founded!")
            node = self.dict[key]
            self.top(node)
            return node.value     

    def put(self, key: int, value: int) -> None:
        # print("______")
        # print("Putting %d" % key)
        # print("Before Operation")
        # print("current size is %d" % self.size)
        # print(self.dict.keys())
        # print("+++++++")
        node = DNode(key,value)
        if key not in self.dict:
            if self.size >= self.capacity:
                del self.dict[self.pop()]
                self.size -= 1
            self.dict[key] = self.push(node)
            self.size += 1
            # print("current size is %d" % self.size)
            # print(self.dict.keys())
        else:
            self.top(self.dict[key])
            self.dict[key].value = value
            # print("current size is %d" % self.size)
            # print(self.dict.keys())
    def __call__(self,func,*args,**kwargs):
        def wrapper(*args, **kwargs):
            key = args[0]
            v = self.get(key)
            start_time = time.time()
            if v == -1:
                v = func(*args,**kwargs);
                self.put(key,v)
                print("[%.8fs] %s(%s) -> %s " % (time.time() - start_time,func.__name__,key,v))
            else:
                print("[cache_hit] %s(%s) -> %s " % (func.__name__,key,v))
            return v
        return wrapper
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)

In [2]:
@lru_cache(3)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

In [3]:
print(f'fibonacci(6)={fibonacci(6)}\n')

[0.00000691s] fibonacci(0) -> 0 
[0.00000381s] fibonacci(1) -> 1 
[0.00009704s] fibonacci(2) -> 1 
[cache_hit] fibonacci(1) -> 1 
[cache_hit] fibonacci(2) -> 1 
[0.00003910s] fibonacci(3) -> 2 
[0.00020671s] fibonacci(4) -> 3 
[cache_hit] fibonacci(3) -> 2 
[cache_hit] fibonacci(4) -> 3 
[0.00026202s] fibonacci(5) -> 5 
[0.00052500s] fibonacci(6) -> 8 
fibonacci(6)=8



In [4]:
INVOKE_COUNT = 0

@lru_cache(4)
def get_data(key):
    global INVOKE_COUNT
    INVOKE_COUNT = INVOKE_COUNT + 1
    return {'id': key, 'value': f'Foo Bar - {key}'}


def test_get_data(keys):
    for x in keys:
        result = get_data(x)
        print(result)
    print(f'Num of function calls:{len(keys)}')

In [5]:
test_get_data([1, 2, 3, 4, 1, 2, 3, 4, 5, 6])
print(f'Num of cache misses:{INVOKE_COUNT}')
assert INVOKE_COUNT == 6

[0.00000882s] get_data(1) -> {'id': 1, 'value': 'Foo Bar - 1'} 
{'id': 1, 'value': 'Foo Bar - 1'}
[0.00000715s] get_data(2) -> {'id': 2, 'value': 'Foo Bar - 2'} 
{'id': 2, 'value': 'Foo Bar - 2'}
[0.00000262s] get_data(3) -> {'id': 3, 'value': 'Foo Bar - 3'} 
{'id': 3, 'value': 'Foo Bar - 3'}
[0.00001001s] get_data(4) -> {'id': 4, 'value': 'Foo Bar - 4'} 
{'id': 4, 'value': 'Foo Bar - 4'}
[cache_hit] get_data(1) -> {'id': 1, 'value': 'Foo Bar - 1'} 
{'id': 1, 'value': 'Foo Bar - 1'}
[cache_hit] get_data(2) -> {'id': 2, 'value': 'Foo Bar - 2'} 
{'id': 2, 'value': 'Foo Bar - 2'}
[cache_hit] get_data(3) -> {'id': 3, 'value': 'Foo Bar - 3'} 
{'id': 3, 'value': 'Foo Bar - 3'}
[cache_hit] get_data(4) -> {'id': 4, 'value': 'Foo Bar - 4'} 
{'id': 4, 'value': 'Foo Bar - 4'}
[0.00000811s] get_data(5) -> {'id': 5, 'value': 'Foo Bar - 5'} 
{'id': 5, 'value': 'Foo Bar - 5'}
[0.00000596s] get_data(6) -> {'id': 6, 'value': 'Foo Bar - 6'} 
{'id': 6, 'value': 'Foo Bar - 6'}
Num of function calls:10
Num